In [1]:
# Install whisper if not already installed
#!pip install -q openai-whisper

## Automated YouTube Video Processing
This notebook now uses functionss from .py docs to: download, transcribe, chunk, enrich, embed, and save a YouTube video transcript.

In [2]:
import sys
sys.path.append('../')

**Troubleshooting ImportError:**  
If you see `ImportError: cannot import name 'transcribe_audio' from 'whisper'`,  
make sure you are using the correct import.  
You should use `import whisper` and then call `whisper.transcribe(...)`  
or check the documentation for the correct function name.

In [3]:
from video_pipeline import process_video

# For single video processing - test
process_video(
    video_url="https://www.youtube.com/watch?v=SN-vBnWj6e8",
    audio_path="../data/audio/SN-vBnWj6e8.mp3",
    transcript_path = "../data/transcripts/SN-vBnWj6e8.txt",
    vectorstore_path="../data/vectorstores/SN-vBnWj6e8_faiss"
)
    

[youtube] Extracting URL: https://www.youtube.com/watch?v=SN-vBnWj6e8
[youtube] SN-vBnWj6e8: Downloading webpage
[youtube] SN-vBnWj6e8: Downloading tv client config
[youtube] SN-vBnWj6e8: Downloading tv player API JSON
[youtube] SN-vBnWj6e8: Downloading ios player API JSON
[youtube] SN-vBnWj6e8: Downloading m3u8 information
[info] SN-vBnWj6e8: Downloading 1 format(s): 251
[download] ../data/audio/SN-vBnWj6e8.mp3 has already been downloaded
[ExtractAudio] Not converting audio ../data/audio/SN-vBnWj6e8.mp3; file is already in target format mp3



/Users/adriannadziadyk/Multimodal-AI-YouTube-QA-Bot/notebooks/../utils/embedding.py:15: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embedding_model = OpenAIEmbeddings(model=model)


'✅ Successfully processed and stored 26 documents.'